# Model Creation

## Import Libraries

In [1]:
import os
from operator import itemgetter    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


### Import Data from Pre-Processing
* Missing values HAVE been imputed.
* No PCA performed yet, no 1hot encoding. 


In [2]:
######################
#
# Import Data from PreProcessing
#
#####################

df= pd.read_csv('../_data/operations_imputed_CLEAN_v2.csv', index_col=0)

df.drop(['race'], axis=1, inplace=True)



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76742 entries, 8 to 128030
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   op_id            76742 non-null  int64  
 1   subject_id       76742 non-null  int64  
 2   hadm_id          76742 non-null  int64  
 3   opdate           76742 non-null  int64  
 4   age              76742 non-null  int64  
 5   sex              76742 non-null  object 
 6   weight           76742 non-null  float64
 7   height           76742 non-null  float64
 8   asa              76742 non-null  float64
 9   department       76742 non-null  object 
 10  antype           76742 non-null  object 
 11  icd10_pcs        76742 non-null  object 
 12  category_desc    76742 non-null  object 
 13  desc_short       76742 non-null  object 
 14  category_id      76742 non-null  object 
 15  hr               76742 non-null  float64
 16  pip              76742 non-null  float64
 17  pmean           

### Create the X and y DataFrames

  * create y
  * create X (complete with all the features)
  * drop the features we identified as not meeting impact threshold. 



In [4]:
# When doing a Categorical Model, reinsert 'prolonged_LOS' and instead, drop 'LOS'

## Features to retain are those in X that will be used in training. Exludued features are features such as Operation_ID, Subject_ID..
features_to_retain = ['category_id','age','sex',	'weight',	'height',	'hr',	'pip',	'pmean',	'rr',	'spo2',	'vt',	'chloride',	'creatinine',	'glucose',	'hb',	'hco3',	'lymphocyte',	'platelet',	'potassium',	'sodium',	'total_bilirubin',	'wbc',	'icu_visit',	'or_duration',	'anesth_duration',	'department','antype'] 

## Create the Y, the Target
y = df['LOS']

## Create X the Features for Train/Test/Validate
X = df.drop('LOS', axis=1)
X= X[features_to_retain]

X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76742 entries, 8 to 128030
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category_id      76742 non-null  object 
 1   age              76742 non-null  int64  
 2   sex              76742 non-null  object 
 3   weight           76742 non-null  float64
 4   height           76742 non-null  float64
 5   hr               76742 non-null  float64
 6   pip              76742 non-null  float64
 7   pmean            76742 non-null  float64
 8   rr               76742 non-null  float64
 9   spo2             76742 non-null  float64
 10  vt               76742 non-null  float64
 11  chloride         76742 non-null  float64
 12  creatinine       76742 non-null  float64
 13  glucose          76742 non-null  float64
 14  hb               76742 non-null  float64
 15  hco3             76742 non-null  float64
 16  lymphocyte       76742 non-null  float64
 17  platelet        

In [7]:
###########################################################
#
#  Indentify the columns that need to be either cast as Str or Scaled
#
############################################################

#Category Columns that will need encoding. Cast them as String
COLS_TO_CAST = ['category_id','antype','sex','department']  #When restoring scope to full category list, add cat_id here.
                                                            # Convert the object data type columns to string

X[COLS_TO_CAST] = X[COLS_TO_CAST].astype(str)

# Numerical Columns for Scaling. Filter columns with dtype 'numeric' for scaling later in the Pipleine
COLS_TO_SCALE = X.select_dtypes(include=['int', 'float']).columns


print(f'Category Cols to encode: {COLS_TO_CAST}')
print(f'Numerical Cols to scale: {COLS_TO_SCALE}')
X.info()

Category Cols to encode: ['category_id', 'antype', 'sex', 'department']
Numerical Cols to scale: Index(['age', 'weight', 'height', 'hr', 'pip', 'pmean', 'rr', 'spo2', 'vt',
       'chloride', 'creatinine', 'glucose', 'hb', 'hco3', 'lymphocyte',
       'platelet', 'potassium', 'sodium', 'total_bilirubin', 'wbc',
       'icu_visit', 'or_duration', 'anesth_duration'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 76742 entries, 8 to 128030
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category_id      76742 non-null  object 
 1   age              76742 non-null  int64  
 2   sex              76742 non-null  object 
 3   weight           76742 non-null  float64
 4   height           76742 non-null  float64
 5   hr               76742 non-null  float64
 6   pip              76742 non-null  float64
 7   pmean            76742 non-null  float64
 8   rr               76742 non-null  float64

## Training Split

- Training Set (80% of total): 
  - Used to train the models.
- Validation Set (20% of Traning Set ): 
  - Used to fine-tune hyperparameters, select models, and monitor training progress.  
- Testing Set (20% of total): 
  - Used to evaluate the final model's performance on unseen data and estimate its generalization performance.

In [5]:
from sklearn.model_selection import train_test_split

TEST_SPLIT = .2
TRAINING_SPLIT = 1-TEST_SPLIT
VALIDATION_SPLIT = .2

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=85100)

# Split the Training AGAIN into train and Validate
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=TEST_SPLIT, random_state=85100)

# use X_train and y_train for model training and X_val and y_val for turning.

data_subset_dict = {
    'X_train': X_train,
    'X_validate': X_validate,
    'X_test': X_test,
    'y_train': y_train,
    'y_validate': y_validate,
    'y_test': y_test}

for key, value in data_subset_dict.items():
    shape = value.shape
    print(f"{key} shape: {shape}")


X_train shape: (49114, 27)
X_validate shape: (12279, 27)
X_test shape: (15349, 27)
y_train shape: (49114,)
y_validate shape: (12279,)
y_test shape: (15349,)


## Scale X (Numerical)
* We do NOT scale Y, the target
* We fit the StandardScaler on X_training and then transform both your training and validation sets

In [8]:
##### 
## SCALE X_train and X_validate
#########
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
                ('num', StandardScaler(), COLS_TO_SCALE)
                ],
    remainder='passthrough')  # Leaves the rest of the columns alone

# Fit on the training data
preprocessor.fit(X_train)

# Transform the training and validation data
X_train_scaled = preprocessor.transform(X_train)
X_validate_scaled = preprocessor.transform(X_validate)

# Now X_train_scaled and X_validate_scaled have the specified columns scaled, and the rest are unchanged

## MODEL PIPELINE

### Linear Regression - Baseline

In [14]:
#########################
#
#  SIMPLE LINEAR REGRESSION Pipeline -
#  -- No tuning. 
########################

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

categorical_transform = Pipeline([('one-hot-encode', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))])

preprocessing_df = ColumnTransformer([('categorical', categorical_transform, COLS_TO_CAST)])

pipeline_base = Pipeline([('preprocessing', preprocessing_df),
                          ('model', LinearRegression())])
pipeline_base.fit(X_train, y_train)

y_pred = pipeline_base.predict(X_validate)
r2 = r2_score(y_validate, y_pred)
rmse = mean_squared_error(y_validate, y_pred, squared=False)
print(f'R-squared of base model: {r2}')
print(f"RMSE of the base model: {rmse:.3f}")




R-squared of base model: -2.904623057685253e+18
RMSE of the base model: 9515250466.184


In [11]:
from sklearn import set_config
set_config(display='diagram')
pipeline_base

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('one-hot-encode',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['category_id', 'antype',
                                                   'sex', 'department'])])),
                ('model', LinearRegression())])

### Ensemble Methods - Baselines


In [ ]:
#########################
#
#  Ensemble Pipeline -
#  -- No tuning. 
########################

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

model_list= [LinearRegression(),ExtraTreesRegressor (n_jobs=-1),RandomForestRegressor(n_jobs=-1),XGBRegressor(n_jobs=-1)]
model_names = ["Linear Regression","ExtraTreesRegressor", "Random Forest","XGBRegressor"]

# model_list= [LinearRegression()]
# model_names = ["Linear Regression"]

ModScores = {}

categorical_transform = Pipeline([('one-hot-encode', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))])

preprocessing_df = ColumnTransformer([('categorical', categorical_transform, COLS_TO_CAST)])

for model_names, model in zip(model_names, model_list):
    pipeline_base = Pipeline([('preprocessing', preprocessing_df),
                          ('model', model)])
    pipeline_base.fit(X_train, y_train)

    y_pred = pipeline_base.predict(X_validate)
    
    # Calculate the R-squared value
    r2 = r2_score(y_validate, y_pred)
    rmse = mean_squared_error(y_validate, y_pred, squared=False)
    
    ModScores[model_names] = rmse
    
    print(f"{model}: R2: {r2:.2f}, RMSE: {rmse:.2f}")

print("_"*100)
for key, value in sorted(ModScores.items(), key=itemgetter(1), reverse=False):
    print(f"{key}: RMSE: {value:.3f}")


### Optimize Model with Hyperparameter Tuning via Grid Search


In [ ]:
#########################
#
#  STANDALONE TUNING  - CatBoost
# 
########################
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from tqdm import tqdm  # Import tqdm
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from catboost import Pool, CatBoostRegressor

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


categorical_features_indices =[0, 2, 25, 26]


# Define the parameter grid to search
param_grid = {
    'iterations': [100, 200, 300],      # Number of boosting iterations
    'depth': [6, 8, 10],                # Depth of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate  
    }

# Create a CatBoostRegressor model
catboost_model = CatBoostRegressor()

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, cv=5, scoring=rmse_scorer, n_jobs=-1, error_score='raise')

# Perform the grid search
grid_search.fit(X_train, y_train, cat_features=categorical_features_indices)

# Print the best hyperparameters and corresponding MSE score
print("Best hyperparameters found:")
print(grid_search.best_params_)
print("Best RMSE score:", -grid_search.best_score_)

# Get the best trained model
best_catboost_model = grid_search.best_estimator_

# Evaluate the best model on the validation set
validation_predictions = best_catboost_model.predict(X_validate)

# Output - best settings for training the model


In [17]:
#########################
#
#  OPTIMIZED - CatBoost
# 
########################

from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


###############
# Specify categorical feature indices
categorical_features_indices = [0, 2, 25, 26]


# Create the Pool for training data
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)

# If you have a validation dataset
validation_pool = Pool(data=X_validate, label=y_validate, cat_features=categorical_features_indices)

#########
# Best hyperparameters found (after hyperparameter tuning)
best_params = {'depth': 8, 'iterations': 200, 'learning_rate': 0.1}

# Instantiate CatBoostRegressor with the best hyperparameters
cat_model = CatBoostRegressor(**best_params)

cat_model.fit(
    train_pool,
    eval_set=validation_pool,  # Remove this if you don't have a validation set
    verbose=10,  # This will print the progress every 10 iterations
    plot=True    # This will plot the learning curve (only works in Jupyter notebooks)
)


predictions = cat_model.predict(X_validate)  # If you used Pool, the data here should not be the Pool object but raw data.


# Calculate the R-squared value
r2 = r2_score(y_validate, predictions)
rmse = mean_squared_error(y_validate, predictions, squared=False)


print(f'R-squared of base model: {r2}')
print(f"RMSE of the base model: {rmse:.3f}")


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 5.4924213	test: 5.3998622	best: 5.3998622 (0)	total: 212ms	remaining: 42.2s
10:	learn: 4.3908041	test: 4.3521760	best: 4.3521760 (10)	total: 909ms	remaining: 15.6s
20:	learn: 4.0655363	test: 4.1091377	best: 4.1091377 (20)	total: 1.63s	remaining: 13.9s
30:	learn: 3.9183661	test: 4.0236519	best: 4.0236519 (30)	total: 2.37s	remaining: 12.9s
40:	learn: 3.8108284	test: 3.9646639	best: 3.9646639 (40)	total: 3.17s	remaining: 12.3s
50:	learn: 3.7525718	test: 3.9464708	best: 3.9464708 (50)	total: 3.94s	remaining: 11.5s
60:	learn: 3.7046461	test: 3.9291717	best: 3.9291717 (60)	total: 4.67s	remaining: 10.6s
70:	learn: 3.6728938	test: 3.9209391	best: 3.9209391 (70)	total: 5.37s	remaining: 9.76s
80:	learn: 3.6356634	test: 3.9115113	best: 3.9115113 (80)	total: 6.06s	remaining: 8.9s
90:	learn: 3.5977834	test: 3.9060306	best: 3.9060306 (90)	total: 6.8s	remaining: 8.14s
100:	learn: 3.5684941	test: 3.9048493	best: 3.9048493 (100)	total: 7.58s	remaining: 7.42s
110:	learn: 3.5428223	test: 3.8949